# Problem Set 7: Text Analysis of DOJ Press Releases, Part 2

0.0 Import packages

In [1]:
## helpful packages
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
import random
import re
import string

## Install relevant packages if you haven't already
## ! conda activate qss20
## ! conda install nltk spacy vaderSentiment gensim scikit-learn "scipy<1.13"

## nltk imports
import nltk
### uncomment and run these lines if you haven't downloaded relevant nltk add-ons yet
### nltk.download('averaged_perceptron_tagger')
### nltk.download('stopwords')
from nltk import pos_tag
from nltk.tokenize import word_tokenize, wordpunct_tokenize
from nltk.stem.snowball import SnowballStemmer
from nltk.corpus import stopwords

## spacy imports
import spacy
### uncomment and run the below line if you haven't loaded the en_core_web_sm library yet
### ! python -m spacy download en_core_web_sm
import en_core_web_sm
nlp = en_core_web_sm.load()

## vectorizer
from sklearn.feature_extraction.text import CountVectorizer

## sentiment
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

## LDA
from gensim import corpora
import gensim

## repeated printouts and wide-format text
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
pd.set_option('display.max_colwidth', None)

0.1 Load and clean text data

In [2]:
## first, unzip the file pset7_inputdata.zip 
## then, run this code to load the unzipped json file and convert to a dataframe
## (may need to change the pathname depending on where you store stuff)
## and convert some of the attributes from lists to values
doj = pd.read_json("pset7_inputdata/combined.json", lines = True)

## due to json, topics are in a list so remove them and concatenate with ;
doj['topics_clean'] = ["; ".join(topic) 
                      if len(topic) > 0 else "No topic" 
                      for topic in doj.topics]

## similarly with components
doj['components_clean'] = ["; ".join(comp) 
                           if len(comp) > 0 else "No component" 
                           for comp in doj.components]

## drop older columns from data
doj = doj[['id', 'title', 'contents', 'date', 'topics_clean', 
           'components_clean']].copy()

doj.head()

id  \
0     None   
1  12-919    
2  11-1002   
3   10-015   
4   18-898   

                                                                                                          title  \
0                                                                  Convicted Bomb Plotter Sentenced to 30 Years   
1                              $1 Million in Restitution Payments Announced to Preserve North Carolina Wetlands   
2                 $1 Million Settlement Reached for Natural Resource Damages at Superfund Site in Massachusetts   
3                                          10 Las Vegas Men Indicted \r\nfor Falsifying Vehicle Emissions Tests   
4  $100 Million Settlement Will Speed Cleanup Work at Centredale Manor Superfund Site in North Providence, R.I.   

                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                

# 1. Topic modeling (25 points)

For this question, use the `doj_subset_wscores` data (you created in 1.3 of Problem Set 6) that is restricted to civil rights, hate crimes, and project safe childhood and with the sentiment scores added


## 1.1 Preprocess the data by removing stopwords, punctuation, and non-alpha words (5 points)

A. Write a function that:

- Takes in a single raw string in the `contents` column from that dataframe
- Does the following preprocessing steps:

    - Converts the words to lowercase
    - Removes stopwords, adding the custom stopwords in your code cell below to the default stopwords list
    - Only retains alpha words (so removes digits and punctuation)
    - Only retains words 4 characters or longer
    - Uses the snowball stemmer from nltk to stem

- Returns a joined preprocessed string
    
B. Use `apply` or list comprehension to execute that function and create a new column in the data called `processed_text`
    
C. Print the `id`, `contents`, and `processed_text` columns for the following press releases:

id = 16-718 (this case: https://www.seattletimes.com/nation-world/doj-miami-police-reach-settlement-in-civil-rights-case/)

id = 16-217 (this case: https://www.wlbt.com/story/32275512/three-mississippi-correctional-officers-indicted-for-inmate-assault-and-cover-up/)
    
**Resources**:

- Here's code examples for the snowball stemmer: https://www.geeksforgeeks.org/snowball-stemmer-nlp/

In [3]:
custom_doj_stopwords = ["civil", "rights", "division", "department", "justice",
                        "office", "attorney", "district", "case", "investigation", "assistant",
                        "trial", "assistance", "assist"]

In [ ]:
## your code defining a text processing function

# Load the doj_subset_wscores data from pset7 input data
doj_subset_wscores = pd.read_csv("pset7_inputdata/doj_subset_wscores.csv")
print(f"Shape of doj_subset_wscores: {doj_subset_wscores.shape}")
doj_subset_wscores[['id', 'topics_clean', 'compound']].head() # Check

# Define the text preprocessing function
stemmer = SnowballStemmer("english")

# Combine default stopwords with custom stopwords
stop_words = set(stopwords.words('english')).union(set(custom_doj_stopwords))

# Build text preprocessing function
def preprocess_text(raw_string):
    # Tokenize and lowercase
    tokens = word_tokenize(raw_string.lower())
    
    # Filter: alpha only, 4+ chars, not a stopword, then stem
    processed = [
        stemmer.stem(word) for word in tokens
        if word.isalpha() and len(word) >= 4 and word not in stop_words
    ]
    
    return " ".join(processed)

Shape of doj_subset_wscores: (717, 11)


,id,topics_clean,compound
0,17-1235,Civil Rights,-0.9920
1,15-1522,Project Safe Childhood,-0.8910
2,16-213,Project Safe Childhood,-0.7579
3,16-381,Project Safe Childhood,-0.9037
4,14-464,Hate Crimes,-0.9864


In [5]:
## your code executing the function

# Apply function to create a new column processed_text
doj_subset_wscores['processed_text'] = doj_subset_wscores['contents'].apply(preprocess_text)
print(f"Created processed_text column. Shape: {doj_subset_wscores.shape}")

# doj_subset_wscores[['id', 'processed_text']].head(3)

Created processed_text column. Shape: (717, 12)


In [6]:
## your code showing the examples

# Print id, contents, and processed_text for the two press releases
example_ids = ['16-718', '16-217']
examples = doj_subset_wscores[doj_subset_wscores['id'].isin(example_ids)]
examples[['id', 'contents', 'processed_text']]

,id,contents,processed_text
313,16-217,"The Justice Department has reached a comprehensive settlement agreement with the city of Miami and the Miami Police Department (MPD) resolving the Justice Department’s investigation of officer-involved shootings by MPD officers, announced Principal Deputy Assistant Attorney General Vanita Gupta, head of the Justice Department’s Civil Rights Division and U.S. Attorney Wifredo A. Ferrer of the Southern District of Florida. The settlement, which was approved by Miami’s city commission today and will go into effect when the agreement is signed by all parties, resolves claims stemming from the Justice Department’s investigation into officer-involved shootings by MPD officers, which was conducted under the Violent Crime Control and Law Enforcement Act of 1994. The investigation’s findings, issued in July 2013, identified a pattern or practice of excessive use of force through officer-involved shootings in violation of the Fourth Amendment of the Constitution. The city’s compliance with the settlement will be monitored by an independent reviewer, former Tampa, Florida, Police Chief Jane Castor. Under the settlement agreement, the city will implement comprehensive reforms to ensure constitutional policing and support public trust. The settlement agreement is designed to minimize officer-involved shootings and to more effectively and quickly investigate officer-involved shootings that do occur, through measures that include: “This settlement represents a renewed commitment by the city of Miami and Chief Rodolfo Llanes to provide constitutional policing for Miami residents and to protect public safety through sustainable reform,” said Principal Deputy Assistant Attorney General Gupta. “The agreement will help to strengthen the relationship between the MPD and the communities they serve by improving accountability for officers who fire their weapons unlawfully, and provides for community participation in the enforcement of this agreement.” “Today's agreement is the result of a joint effort between the Department of Justice and the City of Miami to ensure that the Miami Police Department continues its efforts to make our community safe while protecting the sacred Constitutional rights of all of our citizens,” said U.S. Attorney Ferrer. “Through oversight and communication, the agreement seeks to make permanent the positive changes that former Chief Orosa and Chief Llanes have made, and we applaud the City Commission’s vote.” The settlement agreement builds upon important reforms implemented by the city since the Justice Department issued its findings, including: The investigation was conducted by attorneys and staff from the Civil Rights Division’s Special Litigation Section and the Civil Division of the U. S. Attorney’s Office of the Southern District of Florida.",reach comprehens settlement agreement citi miami miami polic resolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim shoot effect quick investig shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut citizen said ferrer oversight communic agreement seek make perman posit 

## 1.2 Create a document-term matrix from the preprocessed press releases and to explore top words (5 points)

A. Use the `create_dtm` function I provide (alternately, feel free to write your own!) and create a document-term matrix using the preprocessed press releases; make sure metadata contains the following columns: `id`, `compound` sentiment column you added, and the `topics_clean` column

B. Print the top 10 words for press releases with compound sentiment in the top 5% (so the most positive sentiment)

C. Print the top 10 words for press releases with compound sentiment in the bottom 5% (so the most negative sentiment)

**Hint**: for these, remember the pandas quantile function from pset one.  

D. Print the top 10 words for press releases in each of the three `topics_clean`

For steps B - D, to receive full credit, write a function `get_topwords` that helps you avoid duplicated code when you find top words for the different subsets of the data. There are different ways to structure it but one way is to feed it subsetted data (so data subsetted to one topic etc.) and for it to get the top words for that subset.


In [7]:
# Small modification from original given function

def create_dtm(list_of_strings, metadata):
    vectorizer = CountVectorizer(lowercase = True)
    dtm_sparse = vectorizer.fit_transform(list_of_strings)
    dtm_dense_named = pd.DataFrame(dtm_sparse.todense(), 
        columns=vectorizer.get_feature_names_out()) # Changed to _out() for newer sklearn version
    dtm_dense_named_withid = pd.concat([metadata.reset_index(), dtm_dense_named], axis = 1)
    return(dtm_dense_named_withid)

In [8]:
# your code here

# Create a document-term matrix with metadata columns: id, compound, topics_clean
metadata = doj_subset_wscores[['id', 'compound', 'topics_clean']].copy()
dtm = create_dtm(doj_subset_wscores['processed_text'].tolist(), metadata)
print(f"DTM shape: {dtm.shape}")

# Define get_topwords function to avoid duplicated code
# Takes a subsetted DTM dataframe and returns the top n words by total frequency. (Exclides metadata columns)
def get_topwords(dtm_subset, n=10):
    meta_cols = ['id', 'compound', 'topics_clean', 'index']
    word_cols = [c for c in dtm_subset.columns if c not in meta_cols]
    word_sums = dtm_subset[word_cols].sum().sort_values(ascending=False)
    return word_sums.head(n)

# Top 10 words for press releases with compound sentiment in top 5% (most positive)
top_5_threshold = doj_subset_wscores['compound'].quantile(0.95)

# Use the original dataframe's compound column to create the boolean mask
mask_top5 = (doj_subset_wscores['compound'].values >= top_5_threshold)
dtm_top5 = dtm[mask_top5.tolist()]
print(f"\n--- Top 10 words for MOST POSITIVE sentiment (top 5%, compound >= {top_5_threshold:.4f}) ---")
print(f"Number of documents: {len(dtm_top5)}")
get_topwords(dtm_top5)

# Top 10 words for press releases with compound sentiment in bottom 5% (most negative)
bottom_5_threshold = doj_subset_wscores['compound'].quantile(0.05)
mask_bottom5 = (doj_subset_wscores['compound'].values <= bottom_5_threshold)
dtm_bottom5 = dtm[mask_bottom5.tolist()]
print(f"\n--- Top 10 words for MOST NEGATIVE sentiment (bottom 5%, compound <= {bottom_5_threshold:.4f}) ---")
print(f"Number of documents: {len(dtm_bottom5)}")
get_topwords(dtm_bottom5)

# Top 10 words for each of the three topics_clean
for topic in doj_subset_wscores['topics_clean'].unique():
    mask_topic = (doj_subset_wscores['topics_clean'].values == topic)
    dtm_topic = dtm[mask_topic.tolist()]
    print(f"\n--- Top 10 words for '{topic}' (n={len(dtm_topic)}) ---")
    print(get_topwords(dtm_topic))

DTM shape: (717, 6760)

--- Top 10 words for MOST POSITIVE sentiment (top 5%, compound >= 0.9902) ---
Number of documents: 36


agreement     171
enforc        126
state         116
ensur         110
disabl        102
communiti      87
settlement     87
student        86
general        84
servic         80
dtype: int64


--- Top 10 words for MOST NEGATIVE sentiment (bottom 5%, compound <= -0.9976) ---
Number of documents: 36


assault     178
offic       162
victim      157
crime       156
hate        120
defend      108
sentenc      98
charg        96
feder        95
prosecut     89
dtype: int64


--- Top 10 words for 'Civil Rights' (n=305) ---
offic        627
hous         620
discrimin    541
enforc       531
disabl       509
said         497
feder        475
violat       470
state        443
general      408
dtype: int64

--- Top 10 words for 'Project Safe Childhood' (n=166) ---
child          1018
exploit         698
sexual          570
safe            476
childhood       472
project         472
pornographi     447
children        416
crimin          404
prosecut        374
dtype: int64

--- Top 10 words for 'Hate Crimes' (n=246) ---
victim      590
crime       533
prosecut    476
hate        472
defend      459
sentenc     455
charg       452
guilti      430
feder       426
said        424
dtype: int64


## 1.3 Estimate a topic model using those preprocessed words (5 points)

A. Going back to the preprocessed words from part 2.3.1, estimate a topic model with 3 topics, since you want to see if the unsupervised topic models recover different themes for each of the three manually-labeled areas (civil rights; hate crimes; project safe childhood). You have free rein over the other topic model parameters beyond the number of topics.

B. After estimating the topic model, print the top 15 words in each topic.

**Hints and Resources**:

- Same topic modeling resources linked to above
- Make sure to use the `random_state` argument within the model so that the numbering of topics does not move around between runs of your code

In [9]:
# your code here

## Estimate a topic model with 3 topics using the preprocessed words

# Tokenize the processed text into lists of words for gensim
texts = [doc.split() for doc in doj_subset_wscores['processed_text'].tolist()]

# Create dictionary and corpus for gensim
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Estimate LDA model with 3 topics
lda_model = gensim.models.LdaModel(
    corpus=corpus,
    id2word=dictionary,
    num_topics=3,
    random_state=42,
    passes=10,
    alpha='auto',
    per_word_topics=True
)

# B. Print the top 15 words in each topic
print("Top 15 words per topic:\n")
for idx, topic in lda_model.print_topics(num_topics=3, num_words=15):
    print(f"Topic {idx}: {topic}\n")

Top 15 words per topic:

Topic 0: 0.011*"victim" + 0.010*"charg" + 0.009*"sentenc" + 0.009*"prosecut" + 0.009*"defend" + 0.009*"guilti" + 0.008*"feder" + 0.008*"crime" + 0.008*"said" + 0.007*"indict" + 0.007*"hate" + 0.007*"prison" + 0.006*"assault" + 0.006*"investig" + 0.006*"year"

Topic 1: 0.026*"child" + 0.017*"exploit" + 0.014*"sexual" + 0.012*"safe" + 0.012*"project" + 0.012*"childhood" + 0.011*"pornographi" + 0.011*"crimin" + 0.010*"children" + 0.010*"prosecut" + 0.009*"sentenc" + 0.009*"victim" + 0.008*"ceo" + 0.008*"minor" + 0.007*"state"

Topic 2: 0.011*"hous" + 0.010*"discrimin" + 0.009*"disabl" + 0.008*"enforc" + 0.007*"agreement" + 0.007*"said" + 0.006*"state" + 0.006*"court" + 0.006*"alleg" + 0.006*"offic" + 0.006*"requir" + 0.006*"feder" + 0.006*"settlement" + 0.006*"violat" + 0.006*"fair"



## 1.4 Add topics back to main data and explore correlation between manual labels and our estimated topics (10 points)

A. Extract the document-level topic probabilities. Within `get_document_topics`, use the argument `minimum_probability` = 0 to make sure all 3 topic probabilities are returned. Write an assert statement to make sure the length of the list is equal to the number of rows in the `doj_subset_wscores` dataframe

B. Add the topic probabilities to the `doj_subset_wscores` dataframe as columns and create a column, `top_topic`, that reflects each document to its highest-probability topic (eg topic 1, 2, or 3)

C. For each of the manual labels in `topics_clean` (Hate Crime, Civil Rights, Project Safe Childhood), print the breakdown of the % of documents with each top topic (so, for instance, Hate Crime has 246 documents-- if 123 of those documents are coded to topic_1, that would be 50%; and so on). **Hint**: pd.crosstab and normalize may be helpful: https://pandas.pydata.org/pandas-docs/version/0.23/generated/pandas.crosstab.html

D. Using a couple press releases as examples, write a 1-2 sentence interpretation of why some of the manual topics map on more cleanly to an estimated topic than other manual topic(s)


In [10]:
## your code here to get doc-level topic probabilities 

# Extract document-level topic probabilities
doc_topics = [lda_model.get_document_topics(doc, minimum_probability=0) for doc in corpus]

# Assert length matches
assert len(doc_topics) == len(doj_subset_wscores), \
    f"Length mismatch: {len(doc_topics)} topics vs {len(doj_subset_wscores)} documents"
print(f"Number of document-topic distributions: {len(doc_topics)}")
print(f"Number of rows in doj_subset_wscores: {len(doj_subset_wscores)}")
print("Assert passed: lengths match!")

Number of document-topic distributions: 717
Number of rows in doj_subset_wscores: 717
Assert passed: lengths match!


In [11]:
## your code here to add those topic probabilities to the dataframe

# Add topic probabilities to dataframe and create top_topic column

# Extract probabilities for each topic
for topic_num in range(3):
    doj_subset_wscores[f'topic_{topic_num}'] = [doc[topic_num][1] for doc in doc_topics]

# Create top_topic column (topic with highest probability)
topic_cols = ['topic_0', 'topic_1', 'topic_2']
doj_subset_wscores['top_topic'] = doj_subset_wscores[topic_cols].idxmax(axis=1)

print("Topic probability columns added:")
doj_subset_wscores[['id', 'topics_clean', 'topic_0', 'topic_1', 'topic_2', 'top_topic']].head(10)

Topic probability columns added:


,id,topics_clean,topic_0,topic_1,topic_2,top_topic
0,17-1235,Civil Rights,0.999406,0.000292,0.000303,topic_0
1,15-1522,Project Safe Childhood,0.000527,0.999093,0.000381,topic_1
2,16-213,Project Safe Childhood,0.000309,0.999468,0.000223,topic_1
3,16-381,Project Safe Childhood,0.000382,0.999342,0.000276,topic_1
4,14-464,Hate Crimes,0.999333,0.000327,0.000340,topic_0
5,13-1298,Hate Crimes,0.999464,0.000263,0.000273,topic_0
6,17-006,Project Safe Childhood,0.000430,0.999259,0.000311,topic_1
7,17-1335,Project Safe Childhood,0.000399,0.999312,0.000289,topic_1
8,14-973,Hate Crimes,0.999414,0.000288,0.000299,topic_0
9,16-297,Project Safe Childhood,0.000463,0.999203,0.000335,topic_1


In [12]:
## your code here to summarize the topic proportions for each of the topics_clean

# For each manual label, print breakdown of % of documents with each top topic
print("Crosstab of manual topics vs. estimated top topics (normalized by row):\n")
ct = pd.crosstab(doj_subset_wscores['topics_clean'], 
                 doj_subset_wscores['top_topic'], 
                 normalize='index')
ct = ct.round(4) * 100
print(ct)

# Also show raw counts for context
print("\nRaw counts:")
ct_raw = pd.crosstab(doj_subset_wscores['topics_clean'], 
                      doj_subset_wscores['top_topic'])
print(ct_raw)

Crosstab of manual topics vs. estimated top topics (normalized by row):

top_topic               topic_0  topic_1  topic_2
topics_clean                                     
Civil Rights              33.44      0.0    66.56
Hate Crimes               99.19      0.0     0.81
Project Safe Childhood     0.00     99.4     0.60

Raw counts:
top_topic               topic_0  topic_1  topic_2
topics_clean                                     
Civil Rights                102        0      203
Hate Crimes                 244        0        2
Project Safe Childhood        0      165        1


**Interpretation:**
- Project Safe Childhood maps very cleanly to Topic 1, which contains words like "child", "exploit", "sexual", "pornographi", vocabulary that is highly specific to that topic area. 
- Hate Crimes maps primarily to Topic 0, which features "victim", "crime", "hate", "assault" (crime-related language). 
- Civil Rights is mostly captured by Topic 2 with words like "hous", "discrimin", "disabl", related to civil rights enforcement. 
- However, Civil Rights has more overlap with the other topics because its language around prosecution and legal proceedings (e.g., "charg", "sentenc") is shared with Hate Crimes press releases. For example, press releases about civil rights violations that involve criminal charges may use similar language to hate crime prosecutions, blurring the boundary between Topics 0 and 2.

# 2. Extend the analysis from unigrams to bigrams (10 points)

In the previous question, you found top words via a unigram representation of the text. Now, we want to see how those top words change with bigrams (pairs of words)

A. Using the `doj_subset_wscore` data and the `processed_text` column (so the words after stemming/other preprocessing), create a column in the data called `processed_text_bigrams` that combines each consecutive pairs of word into a bigram separated by an underscore. Eg:

"depart reach settlem" would become "depart_reach reach_settlem"

Do this by writing a function `create_bigram_onedoc` that takes in a single `processed_text` string and returns a string with its bigrams structured similarly to above example
 
**Hint**: there are many ways to solve but `zip` may be helpful: https://stackoverflow.com/questions/21303224/iterate-over-all-pairs-of-consecutive-items-in-a-list

B. Print the `id`, `processed_text`, and `processed_text_bigram` columns for press release with id = 16-217

In [ ]:
## your code here 

# Function takes a single processed_text string and returns a string with bigrams separated by underscores
def create_bigram_onedoc(processed_text_string):
    words = processed_text_string.split()
    bigrams = ["_".join(pair) for pair in zip(words[:-1], words[1:])]
    return " ".join(bigrams)

# Apply function to create the bigram column
doj_subset_wscores['processed_text_bigrams'] = doj_subset_wscores['processed_text'].apply(create_bigram_onedoc)

# Print the id, processed_text, and processed_text_bigrams for press release 16-217
example_16_217 = doj_subset_wscores[doj_subset_wscores['id'] == '16-217']
example_16_217[['id', 'processed_text', 'processed_text_bigrams']]

,id,processed_text,processed_text_bigrams
313,16-217,reach comprehens settlement agreement citi miami miami polic resolv shoot offic announc princip deputi general vanita gupta head wifredo ferrer southern florida settlement approv miami citi commiss today effect agreement sign parti resolv claim stem shoot offic conduct violent crime control enforc find issu juli identifi pattern practic excess forc shoot violat fourth amend constitut citi complianc settlement monitor independ review former tampa florida polic chief jane castor settlement agreement citi implement comprehens reform ensur constitut polic support public trust settlement agreement design minim shoot effect quick investig shoot occur measur includ settlement repres renew commit citi miami chief rodolfo llane provid constitut polic miami resid protect public safeti sustain reform said princip deputi general gupta agreement help strengthen relationship communiti serv improv account offic fire weapon unlaw provid communiti particip enforc today agreement result joint effort citi miami ensur miami polic continu effort make communiti safe protect sacr constitut citizen said ferrer oversight communic agreement seek make perman posit chang former chief orosa chief llane made applaud citi commiss settlement agreement build upon import reform implement citi sinc issu find includ conduct attorney staff special litig section southern florida,reach_comprehens comprehens_settlement settlement_agreement agreement_citi citi_miami miami_miami miami_polic polic_resolv resolv_shoot shoot_offic offic_announc announc_princip princip_deputi deputi_general general_vanita vanita_gupta gupta_head head_wifredo wifredo_ferrer ferrer_southern southern_florida florida_settlement settlement_approv approv_miami miami_citi citi_commiss commiss_today today_effect effect_agreement agreement_sign sign_parti parti_resolv resolv_claim claim_stem stem_shoot shoot_offic offic_conduct conduct_violent violent_crime crime_control control_enforc enforc_find find_issu issu_juli juli_identifi identifi_pattern pattern_practic practic_excess excess_forc forc_shoot shoot_violat violat_fourth fourth_amend amend_constitut constitut_citi citi_complianc complianc_settlement settlement_monitor monitor_independ independ_review review_former former_tampa tampa_florida florida_polic polic_chief chief_jane jane_castor castor_settlement settlement_agreement agreement_citi citi_implement implement_comprehens comprehens_reform reform_ensur ensur_constitut constitut_polic polic_support support_public public_trust trust_settlement settlement_agreement agreement_design design_minim minim_shoot shoot_effect effect_quick quick_investig investig_shoot shoot_occur occur_measur measur_includ includ_settlement settlement_repres repres_renew renew_commit commit_citi citi_miami miami_chief chief_rodolfo rodolfo_llane llane_provid provid_constitut constitut_polic polic_miami miami_resid resid_protect protect_public public_safeti safeti_sustain sustain_reform reform_said said_princip princip_deputi deputi_general general_gupta gupta_agreement agreement_help help_strengthen strengthen_relationship relationship_communiti communiti_serv serv_improv improv_account account_offic offic_fire fire_weapon weapon_unlaw unlaw_provid provid_communiti communiti_particip particip_enforc enforc_today today_agreement agreement_result result_joint joint_effort effort_citi citi_miami miami_ensur ensur_miami miami_polic polic_continu continu_effort effort_make make_communiti communiti_safe safe_protect protect_sacr sacr_constitut constitut_citizen citizen_said said_ferrer ferrer_oversight oversight_communic communic_agreement agreement_seek seek_make make_perman perman_posit posit_chang chang_former former_chief chief_orosa orosa_chief chief_llane llane_made made_applaud applaud_citi citi_commiss commiss_settlement settlement_agreement agreement_build build_upon upon_import import_reform reform_implement implement_citi citi_sinc sinc_issu issu_find find

C. Use the create_dtm function and the `processed_text_bigrams` column to create a document-term matrix (`dtm_bigram`) with these bigrams. Keep the following three columns in the data: `id`, `topics_clean`, and `compound` 

D. Print the (1) dimensions of the `dtm` matrix from question 2.2  and (2) the dimensions of the `dtm_bigram` matrix. Comment on why the bigram matrix has more dimensions than the unigram matrix 

E. Find and print the 10 most prevelant bigrams for each of the three topics_clean using the `get_topwords` function from 2.2

In [14]:
# your code here

# Create a DTM using the bigram column
metadata_bigram = doj_subset_wscores[['id', 'topics_clean', 'compound']].copy()
dtm_bigram = create_dtm(doj_subset_wscores['processed_text_bigrams'].tolist(), metadata_bigram)

# Print dimensions of dtm (unigram) and dtm_bigram, with commentary
print(f"Unigram DTM dimensions: {dtm.shape}")
print(f"Bigram DTM dimensions:  {dtm_bigram.shape}")
print(f"\nThe bigram matrix has many more columns than the unigram matrix because the number of possible unique pairs of consecutive words is much larger than the number of unique individual words.")
print("With a vocabulary of V unique words, there could be up to V^2 possible bigrams, so the feature space grows substantially when moving from unigrams to bigrams.")


# Top 10 bigrams for each of the three topics_clean
for topic in doj_subset_wscores['topics_clean'].unique():
    mask_topic = (doj_subset_wscores['topics_clean'].values == topic)
    dtm_topic_bigram = dtm_bigram[mask_topic.tolist()]
    print(f"\n--- Top 10 bigrams for '{topic}' (n={len(dtm_topic_bigram)}) ---")
    print(get_topwords(dtm_topic_bigram))

Unigram DTM dimensions: (717, 6760)
Bigram DTM dimensions:  (717, 71336)

The bigram matrix has many more columns than the unigram matrix because the number of possible unique pairs of consecutive words is much larger than the number of unique individual words.
With a vocabulary of V unique words, there could be up to V^2 possible bigrams, so the feature space grows substantially when moving from unigrams to bigrams.

--- Top 10 bigrams for 'Civil Rights' (n=305) ---
fair_hous         227
deputi_general    221
princip_deputi    221
vanita_gupta      202
gupta_head        200
general_vanita    199
said_princip      186
unit_state        152
nation_origin     141
consent_decre     127
dtype: int64

--- Top 10 bigrams for 'Project Safe Childhood' (n=166) ---
safe_childhood       472
project_safe         472
child_pornographi    445
child_exploit        279
sexual_exploit       223
exploit_children     199
plead_guilti         197
child_sexual         175
exploit_obscen       175
obscen_se